In [3]:
import pycaret

import pandas as pd
import numpy as np

from pycaret.utils import version
version()

'2.3.10'

In [4]:
import json
import time

class Timer:
    def __enter__(self, *args, **kwargs):
        self.tick = time.time()
        return self
    
    def __exit__(self, *args, **kwargs):
        self.elapsed = time.time() - self.tick
        
benchmark_list = []

In [ ]:
# Begin Boiler Plate Code, will have few quick sample commands to run at the end

In [ ]:
# EXAMPLES - Sample Commands